In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, AveragePooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense

In [2]:
train_data = pd.read_csv("train.csv")
train_labels = train_data.pop("label")
labels = np.array(train_labels.values)

labels.shape

(42000,)

In [3]:
def preprocess_train_data(train_data):
    df = train_data.copy()
    tensors = tf.image.grayscale_to_rgb(tf.convert_to_tensor(np.reshape(np.array(df.values), (df.shape[0], 28, 28, 1)), dtype=tf.float32))
    return tensors

In [4]:
features = preprocess_train_data(train_data)

In [12]:
train_X, test_X, train_Y, test_y = train_test_split(features.numpy(),
                                                    labels,
                                                    test_size=0.02,
                                                    random_state=42)
train_Y = tf.one_hot(train_Y, depth = 10, axis=None)
test_Y  = tf.one_hot(test_y, depth = 10)

In [13]:
def create_mnist_model(input_shape, kernel_size=(3,3)):
    model = Sequential()
    model.add(tf.keras.layers.Input(shape=input_shape))
    model.add(tf.keras.layers.Normalization())
    model.add(Conv2D(12, kernel_size, strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), padding='same'))
    model.add(BatchNormalization())

    model.add(Conv2D(24, kernel_size, strides=2, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    model.add(BatchNormalization())
    
    model.add(Conv2D(32, kernel_size, strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    model.add(BatchNormalization())
    
    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())   
    model.add(Dense(64, activation='relu'))
    
    opt = keras.optimizers.Adam()

    model.compile(loss='categorical_crossentropy',
                           optimizer=opt,
                           metrics=['acc'])
    model.add(Dense(10, activation='softmax'))
    model.summary()
    return model
#     print(model.output_shape)
    #add ZEROPAD->CONV-2D->MAXPOOL

In [20]:
def train_and_evaluate(train_X, train_Y, test_X, test_Y):
    input_shape = (28,28,3)
    model = create_mnist_model(input_shape)
    model.fit(train_X, train_Y, epochs=50)
    model.evaluate(test_X, test_Y)
    test_data   = pd.read_csv("test.csv")
    ttx_data = preprocess_train_data(test_data)
    predictions = model.predict(ttx_data)
    predictions = tf.math.argmax(predictions, axis =-1)
    final_data = {'ImageId': list(range(1, predictions.shape[0]+1)), 'Label': np.array(predictions).ravel()}
    submit_df = pd.DataFrame(final_data)
    submit_df.to_csv("submission.csv", index=False)

In [21]:
train_and_evaluate(train_X, train_Y, test_X, test_Y)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization_3 (Normalizati (None, 28, 28, 3)         7         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 26, 26, 12)        336       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 9, 9, 12)          0         
_________________________________________________________________
batch_normalization_12 (Batc (None, 9, 9, 12)          48        
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 5, 5, 24)          2616      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 3, 3, 24)          0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 3, 3, 24)         